In [0]:
path = "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv"

df = spark.read.csv(
    path,
    header=True,
    inferSchema=True
)

df.show(5)
df.printSchema()


+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    NULL|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      view|   1004237|2053013555631882655|electr

In [0]:
df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecommerce_oct2019")


In [0]:
spark.sql("""
CREATE TABLE ecommerce_oct2019_sql
USING DELTA
AS SELECT * FROM ecommerce_oct2019
""")


DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("SHOW TABLES").show()


+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|   ecommerce_oct2019|      false|
| default|ecommerce_oct2019...|      false|
| default|        events_delta|      false|
| default|        events_table|      false|
+--------+--------------------+-----------+



In [0]:
spark.sql("SELECT * FROM ecommerce_oct2019_sql").show(5)


+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    NULL|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      view|   1004237|2053013555631882655|electr

In [0]:
spark.sql("DESCRIBE DETAIL ecommerce_oct2019_sql").show()


+------+--------------------+--------------------+-----------+--------+--------------------+-------------------+----------------+-----------------+--------+-----------+--------------------+----------------+----------------+--------------------+--------------------+-------------+
|format|                  id|                name|description|location|           createdAt|       lastModified|partitionColumns|clusteringColumns|numFiles|sizeInBytes|          properties|minReaderVersion|minWriterVersion|       tableFeatures|          statistics|clusterByAuto|
+------+--------------------+--------------------+-----------+--------+--------------------+-------------------+----------------+-----------------+--------+-----------+--------------------+----------------+----------------+--------------------+--------------------+-------------+
| delta|dd649ea8-1a26-48c...|workspace.default...|       NULL|        |2026-01-12 14:04:...|2026-01-12 14:04:54|              []|               []|      12| 147

In [0]:
from pyspark.sql import Row

wrong_schema_df = spark.createDataFrame(
    [Row(id=2, name="abc", value=1000)]
)

wrong_schema_df.printSchema()


root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- value: long (nullable = true)



In [0]:
try:
    wrong_schema_df.write.format("delta") \
        .mode("append") \
        .saveAsTable("ecommerce_oct2019_sql")
except Exception as e:
    print(f"Schema enforcement: {e}")


Schema enforcement: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: dd649ea8-1a26-48c3-8277-ec1c2bcb3c20).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- event_time: timestamp (nullable = true)
-- event_type: string (nullable = true)
-- product_id: integer (nullable = true)
-- category_id: long (nullable = true)
-- category_code: string (nullable = true)
-- brand: string (nullable = true)
-- price: double (nullable = true)
-- user_id: integer (nullable = true)
-- user_session: string (nullable = true)


Data schema:
root
-- id: long (nullable = true)
-- name: string (nullable = true)
-- value: long (nullable = true)

         
Table ACLs are enabled in this cluster, so auto

In [0]:
spark.sql("DROP TABLE IF EXISTS ecommerce_oct2019_sql")


DataFrame[]